In [ ]:
import sys
import traceback
from importlib import import_module, metadata

VERSIONED = {
    # Core scientific stack
    "numpy": "numpy",
    "matplotlib": "matplotlib",
    "pandas": "pandas",
    "scipy": "scipy",
    "seaborn": "seaborn",

    # ML / DL
    "tensorflow": "tensorflow",
    "tensorflow_model_optimization": "tensorflow-model-optimization",
    "sklearn": "scikit-learn",
    "visualkeras": "visualkeras",

    # Audio
    "librosa": "librosa",

    # Utilities / dev
    "IPython": "ipython",
    "requests": "requests",
    "tqdm": "tqdm",
    "PIL": "pillow",
    "prettytable": "prettytable",
}


IMPORT_ONLY = [
    # stdlib / utils used in notebooks
    "random", "os", "os.path", "pathlib", "json", "zipfile", "tempfile",
    "itertools", "logging", "time", "warnings", "platform", "wave",

    # Jupyter helpers
    "IPython.display",

    # TF Keras modules (already inside TF)
    "tensorflow.keras", "tensorflow.keras.layers", "tensorflow.keras.models",
    "tensorflow.keras.regularizers", "tensorflow.random",

    # librosa submodule
    "librosa.display",

    # sklearn bits used in exercises
    "sklearn.datasets",
    "sklearn.model_selection",
    "sklearn.svm",
    "sklearn.metrics",
    "sklearn.utils.multiclass",
]

EXPECTED = {
    "numpy": "1.24.3",
    "pandas": "2.0.3",
    "matplotlib": "3.7.2",
    "seaborn": "0.12.2",
    "scipy": "1.10.1",
    "tensorflow": "2.13.0",
    "tensorflow_model_optimization": "0.7.5",
    "sklearn": "1.3.2",
    "librosa": "0.10.1",
    "requests": "2.31.0",
    "IPython": "8.12.3",
    "tqdm": "4.66.1",
    "PIL": "10.0.0",
    "prettytable": "3.9.0",
    "visualkeras": "0.0.2"
}

def detect_version(mod_name: str, dist_name: str):
    """Find module version via common attrs then importlib.metadata."""
    try:
        mod = import_module(mod_name)
    except Exception as e:
        return None, f"IMPORT FAILED: {e!r}"

    for attr in ("__version__", "version", "VERSION", "__VERSION__", "__git_version__", "__git_revision__"):
        v = getattr(mod, attr, None)
        if v is not None:
            try:
                return str(v), None
            except Exception:
                pass
    # fallback to distribution metadata
    try:
        return metadata.version(dist_name), None
    except Exception as e:
        return None, f"VERSION UNKNOWN via metadata: {e!r}"

def print_table(rows, headers):
    widths = [max(len(str(x)) for x in col) for col in zip(headers, *rows)]
    fmt = " | ".join(f"{{:<{w}}}" for w in widths)
    sep = "-+-".join("-" * w for w in widths)
    print(fmt.format(*headers))
    print(sep)
    for r in rows:
        print(fmt.format(*r))

def version_audit():
    rows = []
    failures = []
    mismatches = []

    for mod, dist in VERSIONED.items():
        ver, err = detect_version(mod, dist)
        expected = EXPECTED.get(mod)
        match = ("—" if expected is None else (ver == expected))
        status = "OK" if (ver and err is None) else "FAIL"
        rows.append([mod, dist, ver or "-", expected or "—", str(match), status, err or ""])
        if status == "FAIL":
            failures.append(mod)
        elif expected is not None and ver != expected:
            mismatches.append((mod, ver, expected))
    print("\n=== Package Versions ===")
    print_table(rows, headers=["Import", "Distribution", "Detected", "Expected", "Match", "Status", "Note"])

    import_rows = []
    import_fails = []
    for name in IMPORT_ONLY:
        try:
            import_module(name)
            import_rows.append([name, "OK", ""])
        except Exception as e:
            import_rows.append([name, "FAIL", repr(e)])
            import_fails.append(name)
    print("\n=== Import-Only Checks ===")
    print_table(import_rows, headers=["Module", "Status", "Note"])

    print("\n=== Summary ===")
    if failures:
        print(f"- Import/version failures: {len(failures)} → {', '.join(failures)}")
    else:
        print("- No import/version failures in VERSIONED set.")

    if mismatches:
        print(f"- Version mismatches: {len(mismatches)}")
        for m, found, exp in mismatches:
            print(f"  * {m}: detected {found!r} != expected {exp!r}")
    else:
        print("- All detected versions match expected (where specified).")

    if import_fails:
        print(f"- Import-only failures: {len(import_fails)} → {', '.join(import_fails)}")
    else:
        print("- All IMPORT_ONLY modules imported successfully.")

# -----------------------------
# Functional smoke-tests
# -----------------------------
def test_numpy():
    import numpy as np
    a = np.random.randn(4, 3)
    b = np.random.randn(3, 2)
    c = a @ b
    assert c.shape == (4, 2)
    return "OK"

def test_matplotlib():
    import matplotlib
    matplotlib.use("Agg", force=True)
    import matplotlib.pyplot as plt
    import numpy as np
    from PIL import Image, ImageDraw, ImageFont

    fig, ax = plt.subplots()
    ax.plot([0, 0, 1, 1], [0, 1, 1, 0])
    ax.set_title("Matplotlib smoke test")

    text = "Welcome to Machine Learning on\n      Microcontrollers Course"
    center = (0.5, 0.6)      
    width_frac = 0.9         
    dot_size = 3           
    step = 2                 
    jitter = 0.002        
    rng = np.random.default_rng(42)

    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 180)
    except Exception:
        font = ImageFont.load_default()

    tmp = Image.new("L", (1, 1), 0)
    dtmp = ImageDraw.Draw(tmp)
    bbox = dtmp.textbbox((0, 0), text, font=font)        
    w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    pad = int(0.06 * max(w, h))

    img = Image.new("L", (w + 2 * pad, h + 2 * pad), 0)
    draw = ImageDraw.Draw(img)
    draw.text((pad - bbox[0], pad - bbox[1]), text, font=font, fill=255)


    arr = np.array(img)
    ys, xs = np.nonzero(arr > 128)
    if len(xs) > 0 and step > 1:
        xs = xs[::step]
        ys = ys[::step]


    W, H = img.size
    x01 = xs / W
    y01 = 1.0 - (ys / H)  

    aspect = H / W
    text_w = width_frac
    text_h = text_w * aspect
    cx, cy = center
    x_scaled = x01 * text_w + (cx - text_w / 2.0)
    y_scaled = y01 * text_h + (cy - text_h / 2.0)

    if jitter > 0:
        x_scaled = x_scaled + rng.normal(0, jitter, size=x_scaled.shape)
        y_scaled = y_scaled + rng.normal(0, jitter, size=y_scaled.shape)


    ax.scatter(x_scaled, y_scaled, s=dot_size, alpha=0.9, linewidths=0,
               transform=ax.transAxes, zorder=5)

    fig.tight_layout()
    fig.savefig("test_plot.png", dpi=120)
    plt.close(fig)
    return "OK (saved test_plot.png)"


def test_tensorflow_and_visualkeras():
    import tensorflow as tf
    from tensorflow.keras import layers, models
    import numpy as np

    model = models.Sequential([
        layers.Input(shape=(8,)),
        layers.Dense(16, activation="relu"),
        layers.Dropout(0.1),
        layers.Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    _ = model.predict(np.random.randn(5, 8).astype("float32"), verbose=0)

    info = f"TF {tf.__version__}, devices={tf.config.list_physical_devices()}"

    import visualkeras
    from PIL import ImageFont, ImageDraw, Image

    if not hasattr(ImageFont.ImageFont, "getsize"):
        def _getsize(self, text):
            img = Image.new("L", (1, 1))
            draw = ImageDraw.Draw(img)
            l, t, r, b = draw.textbbox((0, 0), text, font=self)
            return r - l, b - t
        ImageFont.ImageFont.getsize = _getsize
        if hasattr(ImageFont, "FreeTypeFont"):
            ImageFont.FreeTypeFont.getsize = _getsize

    try:
        try:
            font = ImageFont.truetype("DejaVuSans.ttf", 24)
        except Exception:
            font = ImageFont.load_default()
        visualkeras.layered_view(model, to_file="model_visualization.png", legend=True, font=font)
        info += " | visualkeras OK (saved model_visualization.png)"
    except Exception as e:
        visualkeras.layered_view(model, to_file="model_visualization.png", legend=False)
        info += f" | visualkeras OK w/o legend (saved model_visualization.png); compat: {type(e).__name__}: {e}"

    return info


def test_tfmot():
    try:
        import tensorflow_model_optimization as tfmot
        _ = dir(tfmot.quantization.keras)  # touch API
        return "OK"
    except Exception as e:
        return f"FAIL: {e!r}"

def test_librosa():
    import numpy as np
    import librosa
    sr = 22050
    t = 0.5  # seconds
    tt = np.linspace(0, t, int(sr * t), endpoint=False)
    y = 0.5 * np.sin(2 * np.pi * 440.0 * tt)  
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    assert mfcc.shape[0] == 13
    return f"OK (mfcc shape={mfcc.shape})"

def test_sklearn():
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    iris = datasets.load_iris()
    X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)
    clf = SVC(kernel="linear", random_state=0)
    clf.fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    assert acc > 0.7
    return f"OK (accuracy={acc:.3f})"

def test_pandas():
    import pandas as pd
    import numpy as np
    df = pd.DataFrame({"a": np.arange(5), "b": np.arange(5) ** 2})
    desc = df.describe().to_dict()
    assert "a" in desc and "b" in desc
    return "OK"

def test_tqdm():
    import sys
    from tqdm import tqdm
    for _ in tqdm(range(50), desc="tqdm smoke", leave=False, file=sys.stdout):
        pass
    return "OK"



def test_all():
    version_audit()

    tests = [
        ("NumPy", test_numpy),
        ("Matplotlib", test_matplotlib),
        ("TensorFlow + visualkeras", test_tensorflow_and_visualkeras),
        ("TF Model Optimization (tfmot)", test_tfmot),
        ("librosa", test_librosa),
        ("scikit-learn", test_sklearn),
        ("pandas", test_pandas),
        ("tqdm", test_tqdm)
    ]

    print("\n=== Functional Smoke-Tests ===")
    results = []
    failures = 0
    for name, fn in tests:
        try:
            out = fn()
            ok = "OK" in out and "FAIL" not in out
            if not ok:
                failures += 1
            results.append([name, "PASS" if ok else "FAIL", out])
        except Exception as e:
            failures += 1
            results.append([name, "FAIL", f"{e.__class__.__name__}: {e}"])
            #traceback.print_exc()

    print_table(results, headers=["Test", "Result", "Details"])

    print("\n=== Final Status ===")
    if failures == 0:
        print("ALL TESTS PASSED")
        rc = 0
    else:
        print(f"{failures} TEST(S) FAILED — see details above.")
        rc = 1




test_all()